In [96]:
from rdkit import Chem
import pandas as pd
import numpy as np

from IPython.display import SVG
from rdkit.Chem import Draw

import json
import re

In [2]:
jnatprod = pd.read_csv('data/entities_img_dataframe_jnatprod.tsv', sep='\t')  
metabo = pd.read_csv('data/entities_img_dataframe_metabo.tsv', sep='\t')
phytorev = pd.read_csv('data/entities_img_dataframe_phytorev.tsv', sep='\t')  
qn = pd.read_csv('data/entities_img_dataframe_qn.tsv', sep='\t')
chrom_a = pd.read_csv('data/entities_img_dataframe_chroma_a.tsv', sep='\t')
chrom_b = pd.read_csv('data/entities_img_dataframe_chroma_b.tsv', sep='\t')

gn_jnatprod = pd.read_csv('data/gn_dataframe_jnatprod.tsv', sep='\t')  
gn_metabo = pd.read_csv('data/gn_dataframe_metabo.tsv', sep='\t')  
gn_phytorev = pd.read_csv('data/gn_dataframe_phytorev.tsv', sep='\t')  
gn_qn = pd.read_csv('data/gn_dataframe_qn.tsv', sep='\t')
gn_chrom_a = pd.read_csv('data/gn_dataframe_chroma_a.tsv', sep='\t')
gn_chrom_b = pd.read_csv('data/gn_dataframe_chroma_b.tsv', sep='\t')

/home/rsilva/miniconda3/envs/nplibrary/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (1,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/rsilva/miniconda3/envs/nplibrary/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
gn = pd.concat([gn_jnatprod, gn_metabo,
                gn_phytorev, gn_qn, gn_chrom_a, gn_chrom_b])

entity = pd.concat([jnatprod, metabo, phytorev, qn, chrom_a, chrom_b])

entity.head()

,doi,smiles,standardInChIKey,ExactMolWt,pubchem,standardInChI,source
0,np50051a041,NaN,MBMQEIFVQACCCH-QBODLPLBSA-N,318.146724,5.28158e+06,InChI=1S/C18H22O5/c1-12-6-5-9-14(19)8-4-2-3-7-...,oscar
1,np50051a041,NaN,RGSFGYAAUTVSQA-UHFFFAOYSA-N,70.078250,9253,InChI=1S/C5H10/c1-2-4-5-3-1/h1-5H2,oscar
2,np50051a041,NaN,OKTJSMMVPCPJKN-UHFFFAOYSA-N,12.000000,5.46231e+06,InChI=1S/C,oscar
3,np50051a041,NaN,BUGBHKTXTAQXES-UHFFFAOYSA-N,79.916521,6.32697e+06,InChI=1S/Se,oscar
4,np50051a041,NaN,OKTJSMMVPCPJKN-UHFFFAOYSA-N,12.000000,5.46231e+06,InChI=1S/C,oscar


In [4]:
gn.fillna('', inplace=True)
orch = gn[gn['classificationPath'].str.contains('Orchidaceae')]
orch.head()

,doi,verbatim,odds,dataSourceId,taxonId,classificationPath,classificationRank,matchType
2049,np000030f_gn.txt,Sarcophyton,8.553086e+03,1,55014845,Plantae|Tracheophyta|Liliopsida|Asparagales|Or...,kingdom|phylum|class|order|family|genus,ExactMatch
4425,np960174n_gn.txt,Sarcophyton,8.553086e+03,1,55014845,Plantae|Tracheophyta|Liliopsida|Asparagales|Or...,kingdom|phylum|class|order|family|genus,ExactMatch
4429,np960174n_gn.txt,Sarcophyton,8.553086e+03,1,55014845,Plantae|Tracheophyta|Liliopsida|Asparagales|Or...,kingdom|phylum|class|order|family|genus,ExactMatch
4799,np50074a006_gn.txt,COELOGYNE,8.571471e+04,1,54840775,Plantae|Tracheophyta|Liliopsida|Asparagales|Or...,kingdom|phylum|class|order|family|genus,ExactMatch
4802,np50074a006_gn.txt,"Coelogyne ochracea,",3.096601e+11,1,43018096,Plantae|Tracheophyta|Liliopsida|Asparagales|Or...,kingdom|phylum|class|order|family|genus|species,ExactCanonicalMatch


In [5]:
orch['doi'] = orch['doi'].str.replace('_gn.txt', '')
orch.head()

/home/rsilva/miniconda3/envs/nplibrary/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,doi,verbatim,odds,dataSourceId,taxonId,classificationPath,classificationRank,matchType
2049,np000030f,Sarcophyton,8.553086e+03,1,55014845,Plantae|Tracheophyta|Liliopsida|Asparagales|Or...,kingdom|phylum|class|order|family|genus,ExactMatch
4425,np960174n,Sarcophyton,8.553086e+03,1,55014845,Plantae|Tracheophyta|Liliopsida|Asparagales|Or...,kingdom|phylum|class|order|family|genus,ExactMatch
4429,np960174n,Sarcophyton,8.553086e+03,1,55014845,Plantae|Tracheophyta|Liliopsida|Asparagales|Or...,kingdom|phylum|class|order|family|genus,ExactMatch
4799,np50074a006,COELOGYNE,8.571471e+04,1,54840775,Plantae|Tracheophyta|Liliopsida|Asparagales|Or...,kingdom|phylum|class|order|family|genus,ExactMatch
4802,np50074a006,"Coelogyne ochracea,",3.096601e+11,1,43018096,Plantae|Tracheophyta|Liliopsida|Asparagales|Or...,kingdom|phylum|class|order|family|genus|species,ExactCanonicalMatch


In [6]:
orch.shape

(1356, 8)

In [7]:
orch_entity = entity[entity['doi'].isin(orch['doi'])]

In [8]:
len(orch_entity['pubchem'].unique())

5557

In [10]:
bioactivity_outcome = pd.read_csv('data/pubchem_bioactivity_results.tsv', sep='\t')
bioactivity_outcome.head()

,AID,AID Version,AID Revision,Panel Member ID,SID,CID,Bioactivity Outcome,Target GI,Target GeneID,Activity Value [uM],Activity Name,Assay Name,Bioassay Type,PubMed ID,RNAi
0,357,2,1,NaN,11111663,1045,Inactive,NaN,NaN,NaN,NaN,AP1 Signaling Pathway,Confirmatory,NaN,NaN
1,410,1,5,NaN,11111663,1045,Inactive,73915100.0,1544.0,NaN,NaN,p450-cyp1a2,Confirmatory,NaN,NaN
2,411,2,1,NaN,11111663,1045,Inactive,160794.0,116160065.0,NaN,Potency,qHTS Assay for Inhibitors of Firefly Luciferase,Confirmatory,NaN,NaN
3,444,1,1,NaN,11111663,1045,Inactive,NaN,NaN,NaN,NaN,NFAT Signaling Pathway,Confirmatory,NaN,NaN
4,445,3,1,NaN,11111663,1045,Inactive,10092619.0,4792.0,NaN,NaN,IkB Signaling,Confirmatory,NaN,NaN


In [11]:
bioactivity_orch = bioactivity_outcome[bioactivity_outcome['CID'].isin(orch_entity['pubchem'].unique())]
bioactivity_orch.shape

/home/rsilva/miniconda3/envs/nplibrary/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(1218674, 15)

In [12]:
len(bioactivity_orch['CID'].unique())

2541

In [13]:
bioactivity_orch['Bioassay Type'].value_counts()

Confirmatory    660170
Other           239227
Screening       226710
Summary          92567
Name: Bioassay Type, dtype: int64

In [14]:
orch_entity.head()

,doi,smiles,standardInChIKey,ExactMolWt,pubchem,standardInChI,source
1387,np200766d,NaN,HEDRZPFGACZZDS-MICDWDOJSA-N,118.920660,71583,InChI=1S/CHCl3/c2-1(3)4/h1H/i1D,oscar
1388,np200766d,NaN,CSCPPACGZOOCGX-UHFFFAOYSA-N,58.041865,180,InChI=1S/C3H6O/c1-3(2)4/h1-2H3,oscar
1389,np200766d,NaN,MJQHZNBUODTQTK-WKGBVCLCSA-N,630.200729,1.19663e+07,InChI=1S/C24H38O19/c25-1-5-9(27)11(29)12(30)22...,oscar
1390,np200766d,NaN,QVGXLLKOCUKJST-UHFFFAOYSA-N,15.994915,159832,InChI=1S/O,oscar
1391,np200766d,NaN,JUJWROOIHBZHMG-UHFFFAOYSA-N,79.042199,1049,InChI=1S/C5H5N/c1-2-4-6-5-3-1/h1-5H,oscar


In [15]:
orch_entity[(orch_entity['source']=='osra') & 
            (orch_entity['pubchem'].isin(bioactivity_orch['CID'].unique()))].shape
                

(616, 7)

In [16]:
orch_sel = orch_entity[(orch_entity['source']=='osra') & 
                       (orch_entity['pubchem'].isin(bioactivity_orch.loc[bioactivity_orch['Bioassay Type']=='Confirmatory',
                                                                     'CID'].unique()))]

In [17]:
orch_sel.shape

(474, 7)

In [18]:
len(orch_sel['pubchem'].unique())

382

In [19]:
orch_sel = orch_sel[~orch_sel['pubchem'].duplicated()]

In [20]:
orch_sel.head()

,doi,smiles,standardInChIKey,ExactMolWt,pubchem,standardInChI,source
318782,np800064w,COC(=O)CC1(O)c2nc3ccccc3c(=O)n2c2c1cccc2,CGMNLIDQBYRXFP-UHFFFAOYSA-N,322.095357,2.49706e+07,NaN,osra
318783,np800064w,Oc1cccc2c1C(=O)c1n2c(=O)c2c(n1)cccc2,WOQXYBJCORZEEE-UHFFFAOYSA-N,264.053492,2.49706e+07,NaN,osra
318785,np800064w,COC(=O)c1c2[nH]c3ccccc3c(=O)n2c2c1cccc2,SJEKGIWFCNGPHF-UHFFFAOYSA-N,292.084792,2.49706e+07,NaN,osra
318786,np800064w,COC(=O)C1(O)c2ccccc2N=C2N1c1ccccc1C2=O,KBIGHBLKYNEVDZ-UHFFFAOYSA-N,308.079707,384867,NaN,osra
318787,np800064w,O=C1c2nc3ccccc3c(=O)n2c2c1cccc2,VQQVWGVXDIPORV-UHFFFAOYSA-N,248.058577,73549,NaN,osra


In [36]:
orch_sel.shape

(382, 7)

In [22]:
import os
if not os.path.exists('figs'):
    os.mkdir('figs')

In [27]:
from rdkit.Chem import Draw

for i in orch_sel.index:
    ms = Chem.MolFromSmiles(orch_sel.loc[i, 'smiles'])
    Draw.MolToFile(ms,'figs/%s.png' % orch_sel.loc[i, 'pubchem'])

In [65]:
orch_print = orch_sel[['doi', 'pubchem', 'ExactMolWt', 'smiles']]
orch_print.reset_index(drop=True, inplace=True)
orch_print.head()

,doi,pubchem,ExactMolWt,smiles
0,np800064w,2.49706e+07,322.095357,COC(=O)CC1(O)c2nc3ccccc3c(=O)n2c2c1cccc2
1,np800064w,2.49706e+07,264.053492,Oc1cccc2c1C(=O)c1n2c(=O)c2c(n1)cccc2
2,np800064w,2.49706e+07,292.084792,COC(=O)c1c2[nH]c3ccccc3c(=O)n2c2c1cccc2
3,np800064w,384867,308.079707,COC(=O)C1(O)c2ccccc2N=C2N1c1ccccc1C2=O
4,np800064w,73549,248.058577,O=C1c2nc3ccccc3c(=O)n2c2c1cccc2


In [66]:
orch_print['Structure'] = ''

/home/rsilva/miniconda3/envs/nplibrary/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
with open('data/doi_dnp.json') as f:
    jnp_doi = json.load(f)
    
with open('data/doi_quim.json') as f:
    qn_doi = json.load(f)

with open('data/doi_metabo.json') as f:
    metabo_doi = json.load(f)

with open('data/doi_phytorev.json') as f:
    pytho_doi = json.load(f)

with open('data/doi_chroma_a.json') as f:
    chroma_a_doi = json.load(f)

with open('data/doi_chroma_b.json') as f:
    chroma_b_doi = json.load(f)

In [67]:
jnp_doi[:5]

['https://pubs.acs.org/doi/pdf/10.1021/np50001a700',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a001',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a002',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a003',
 'https://pubs.acs.org/doi/pdf/10.1021/np50001a004']

In [68]:
metabo_doi[:5]

['https://link.springer.com/content/pdf/10.1007%2Fs11306-005-1111-7.pdf',
 'https://link.springer.com/content/pdf/10.1007%2Fs11306-005-1102-8.pdf',
 'https://link.springer.com/content/pdf/10.1007%2Fs11306-005-1103-7.pdf',
 'https://link.springer.com/content/pdf/10.1007%2Fs11306-005-1104-6.pdf',
 'https://link.springer.com/content/pdf/10.1007%2Fs11306-005-1105-5.pdf']

In [69]:
dois = pd.DataFrame(jnp_doi+qn_doi+metabo_doi+pytho_doi+chroma_b_doi+chroma_a_doi)
dois.head()

,0
0,https://pubs.acs.org/doi/pdf/10.1021/np50001a700
1,https://pubs.acs.org/doi/pdf/10.1021/np50001a001
2,https://pubs.acs.org/doi/pdf/10.1021/np50001a002
3,https://pubs.acs.org/doi/pdf/10.1021/np50001a003
4,https://pubs.acs.org/doi/pdf/10.1021/np50001a004


In [70]:
orch_print.loc[0, 'doi']

'np800064w'

In [71]:
dois.loc[dois[0].str.contains(orch_print.loc[0, 'doi']), 0].values[0]

'https://pubs.acs.org/doi/pdf/10.1021/np800064w'

In [72]:
for i in orch_print.index:
    link = dois.loc[dois[0].str.contains(orch_print.loc[i, 'doi']), 0].values[0]
    orch_print.loc[i, 'doi'] = '<a href="{}" target="_blank">{}</a>'.format(*[link, orch_print.loc[i, 'doi']])    
    orch_print.loc[i, 'Structure'] = '<img src="./figs/%s.png" width="100" height="120">' % orch_print.loc[i, 'pubchem']
    orch_print.loc[i, 'pubchem'] = '<a href="https://pubchem.ncbi.nlm.nih.gov/compound/{}" target="_blank">{}</a>'.format(*[int(orch_print.loc[i, 'pubchem']), int(orch_print.loc[i, 'pubchem'])])
    orch_print.loc[i, 'smiles'] = '<a href="https://pubchem.ncbi.nlm.nih.gov/edit3/index.html?smiles={}" target="_blank">{}</a>'.format(*[orch_print.loc[i, 'smiles'], 'PubChem Sketcher'])

In [73]:
orch_print.head()

,doi,pubchem,ExactMolWt,smiles,Structure
0,"<a href=""https://pubs.acs.org/doi/pdf/10.1021/...","<a href=""https://pubchem.ncbi.nlm.nih.gov/comp...",322.095357,"<a href=""https://pubchem.ncbi.nlm.nih.gov/edit...","<img src=""./figs/24970643.0.png"" width=""100"" h..."
1,"<a href=""https://pubs.acs.org/doi/pdf/10.1021/...","<a href=""https://pubchem.ncbi.nlm.nih.gov/comp...",264.053492,"<a href=""https://pubchem.ncbi.nlm.nih.gov/edit...","<img src=""./figs/24970644.0.png"" width=""100"" h..."
2,"<a href=""https://pubs.acs.org/doi/pdf/10.1021/...","<a href=""https://pubchem.ncbi.nlm.nih.gov/comp...",292.084792,"<a href=""https://pubchem.ncbi.nlm.nih.gov/edit...","<img src=""./figs/24970645.0.png"" width=""100"" h..."
3,"<a href=""https://pubs.acs.org/doi/pdf/10.1021/...","<a href=""https://pubchem.ncbi.nlm.nih.gov/comp...",308.079707,"<a href=""https://pubchem.ncbi.nlm.nih.gov/edit...","<img src=""./figs/384867.0.png"" width=""100"" hei..."
4,"<a href=""https://pubs.acs.org/doi/pdf/10.1021/...","<a href=""https://pubchem.ncbi.nlm.nih.gov/comp...",248.058577,"<a href=""https://pubchem.ncbi.nlm.nih.gov/edit...","<img src=""./figs/73549.0.png"" width=""100"" heig..."


In [74]:
orch_print.tail()

,doi,pubchem,ExactMolWt,smiles,Structure
377,"<a href=""https://www.sciencedirect.com/science...","<a href=""https://pubchem.ncbi.nlm.nih.gov/comp...",192.115030,"<a href=""https://pubchem.ncbi.nlm.nih.gov/edit...","<img src=""./figs/56835056.0.png"" width=""100"" h..."
378,"<a href=""https://www.sciencedirect.com/science...","<a href=""https://pubchem.ncbi.nlm.nih.gov/comp...",162.068080,"<a href=""https://pubchem.ncbi.nlm.nih.gov/edit...","<img src=""./figs/4679494.0.png"" width=""100"" he..."
379,"<a href=""https://www.sciencedirect.com/science...","<a href=""https://pubchem.ncbi.nlm.nih.gov/comp...",108.068748,"<a href=""https://pubchem.ncbi.nlm.nih.gov/edit...","<img src=""./figs/7516.0.png"" width=""100"" heigh..."
380,"<a href=""https://www.sciencedirect.com/science...","<a href=""https://pubchem.ncbi.nlm.nih.gov/comp...",219.104799,"<a href=""https://pubchem.ncbi.nlm.nih.gov/edit...","<img src=""./figs/297591.0.png"" width=""100"" hei..."
381,"<a href=""https://www.sciencedirect.com/science...","<a href=""https://pubchem.ncbi.nlm.nih.gov/comp...",152.083730,"<a href=""https://pubchem.ncbi.nlm.nih.gov/edit...","<img src=""./figs/68126.0.png"" width=""100"" heig..."


In [69]:
# https://pubchem.ncbi.nlm.nih.gov/edit3/index.html?smiles=N1(C)C(=O)N(C)C(C(=C1N1)N(C=1)C)=O
orch_print.to_csv('orch_print.tsv', sep='\t', index=None)

In [102]:
html = '''<!DOCTYPE html>
<html>
  <head>
    <meta http-equiv="content-type" content="text/html; charset=UTF-8">
    <title>NPMINE</title>
    <script src="https://cdn.jsdelivr.net/npm/vue/dist/vue.js"></script>
    <script src="https://code.jquery.com/jquery-3.3.1.js" integrity="sha256-2Kok7MbOyxpgUVvAk/HJ2jigOSYS2auK4Pfzbm7uH60=" crossorigin="anonymous"></script>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js" integrity="sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl" crossorigin="anonymous"></script>
    <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.6.1/css/all.css" integrity="sha384-gfdkjb5BdAXd+lj+gudLWI+BXq4IuLW5IT+brZEZsLFm++aCMlF1V92rMkPaX4PP" crossorigin="anonymous">
</head>

  <body>
    <!-- A grey horizontal navbar that becomes vertical on small screens -->
    <nav class="navbar navbar-expand-sm bg-light navbar-light">
        <a class="navbar-brand" href="">
        </a>

      <!-- Links-->
      <ul class="navbar-nav">
        <li class="nav-item">
          <a class="nav-link" href="http://ccbl.fcfrp.usp.br">CCBL</a>
        </li>
      </ul>

    </nav>
    <div id="content">

    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js" integrity="sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl" crossorigin="anonymous"></script>
    <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.6.1/css/all.css" integrity="sha384-gfdkjb5BdAXd+lj+gudLWI+BXq4IuLW5IT+brZEZsLFm++aCMlF1V92rMkPaX4PP" crossorigin="anonymous">

    <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>

    <script src="https://cdn.datatables.net/1.10.4/js/jquery.dataTables.min.js"></script>
    <link rel="stylesheet" href="https://cdn.datatables.net/1.10.4/css/jquery.dataTables.min.css">
    <script>
    	var dataSet = REPLACE;
        console.log(dataSet);
        $(document).ready(function() {
    	$('#resTable').DataTable( {
            data : dataSet,
    	    // add column definitions to map your json to the table
    	    "columns": [
    		{title: "doi"},
    		{title: "pubchem"},
    		{title: "ExactMolWt"},
    		{title: "smiles"},
            {title: "Structure"}
    	    ]
    	} );
        });
    </script>
    <div class="m-5">
        <table id="resTable" class="table table-striped" style="width:100%" >
		<thead>
	    <tr>
		<th>DOI</th>
		<th>PubChem</th>
		<th>ExactMolWt</th>
		<th>SMILES</th>
		<th>Structure</th>
	    </tr>
	    </thead>
	    </table>
    </div>

    </div>
  </body>
</html>'''

In [103]:
html = re.sub('REPLACE', json.dumps(orch_print.head().apply(lambda a: a.tolist(), axis=1).tolist()), html)

In [104]:
with open('study_case_example.html', 'w+') as f:
    f.write(html)